In [103]:
# | include: false
# | default_exp scilint

In [158]:
# | export

import ast
import os
import re
import shutil
import sys
import warnings
from collections import Counter
from configparser import InterpolationMissingOptionError
from pathlib import Path
from typing import Iterable, Dict, Any

import nbformat
import numpy as np
import pandas as pd
from execnb.nbio import read_nb
from fastcore.script import call_parse
from nbdev.clean import nbdev_clean
from nbdev.config import get_config
from nbdev.doclinks import nbdev_export, nbglob
from nbdev.quarto import nbdev_docs, nbdev_readme
from nbdev.test import nbdev_test
from nbqa.__main__ import _get_configs, _main
from nbqa.cmdline import CLIArgs
from nbqa.find_root import find_project_root

In [105]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test Data Prep

In [106]:
nbdev_path = Path(Path(".").resolve(), "example_nbs", "nbdev.ipynb")
nbdev_hq_path = Path(Path(".").resolve(), "example_nbs", "nbdev_high_quality.ipynb")
non_nbdev_path = Path(Path(".").resolve(), "example_nbs", "non_nbdev.ipynb")
non_nbdev_lq_path = Path(
    Path(".").resolve(), "example_nbs", "non_nbdev_low_quality.ipynb"
)
index_path = Path(Path(".").resolve(), "index.ipynb")
syntax_error_path = Path(Path(".").resolve(), "syntax_error.ipynb")

nbdev_nb = read_nb(nbdev_path)
nbdev_hq_nb = read_nb(nbdev_hq_path)
non_nbdev_nb = read_nb(non_nbdev_path)
non_nbdev_lq_nb = read_nb(non_nbdev_lq_path)
index = read_nb(index_path)
syntax_error = read_nb(index_path)

# `scilint_tidy` 

simple wrapper around no-decisions version of nbqa

In [107]:
# | export


def run_nbqa_cmd(cmd):
    print(f"Running {cmd}")
    project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
    args = CLIArgs.parse_args([cmd, str(project_root)])
    configs = _get_configs(args, project_root)
    output_code = _main(args, configs)
    return output_code

In [108]:
project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
assert os.path.basename(project_root) == "scilint"

In [109]:
# | export


def tidy():
    tidy_tools = ["black", "isort", "autoflake"]
    [run_nbqa_cmd(c) for c in tidy_tools]

# Helpers

In [110]:
# | export


def is_nbdev_project(project_path: Path = Path(".")):
    is_nbdev = True
    project_root = find_project_root(tuple([str(project_path.resolve())]))

    if not Path(project_root, "settings.ini").exists():
        is_nbdev = False
    try:
        get_config().lib_name
    except InterpolationMissingOptionError:
        is_nbdev = False

    return is_nbdev

In [111]:
assert is_nbdev_project()

In [112]:
import tempfile

with tempfile.TemporaryDirectory() as tmp_dir:
    assert not is_nbdev_project(Path(tmp_dir))

In [113]:
# | export


def get_func_defs(code, ignore_private_prefix=True):
    func_names = []
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.FunctionDef):
            inner_cond = (
                False if ignore_private_prefix and stmt.name.startswith("_") else True
            )
            if inner_cond:
                func_names.append(stmt.name)
    return func_names

In [114]:
test_code = """
x()
def y():
    pass
def z():
    def a():
        pass
class A():
    def b():
        pass
def blabla():
    return 1
def _hidden():
    return None
"""
func_defs = ["a", "b", "blabla", "y", "z"]
assert func_defs == sorted(get_func_defs(test_code))

In [115]:
# | export


def count_func_calls(code, func_defs):
    func_calls = Counter({k: 0 for k in func_defs})
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Call):
            func_name = stmt.func.id if "id" in stmt.func.__dict__ else stmt.func.attr
            if func_name in func_defs:
                if func_name in func_calls:
                    func_calls[func_name] += 1
    return func_calls

In [116]:
test_code = """self.hierarchical_topic_reduction(3); 
topic_reduction(3); 
lambda x: topic(x); 
hierarchical_topic_reduction[4]; 
hierarchical_topic_reduction(4); 
blabla()
"""
test_func_defs = [
    "topic",
    "topic_reduction",
    "blablabla",
    "hierarchical_topic_reduction",
]

In [117]:
assert count_func_calls(test_code, test_func_defs) == Counter(
    {
        "topic": 1,
        "topic_reduction": 1,
        "blablabla": 0,
        "hierarchical_topic_reduction": 2,
    }
)

In [118]:
nb_cell_code = r"""
def something():
    pass; pass # in x 2
    
%load_ext autoreload
%autoreload 2

!ls -l
if 1!= 2:
    print(4)
#| export

import pandas as pd # out
from sciflow.utils import lib_path, odbc_connect, query # out

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
nb_to_sagemaker_pipeline() # out
"""

In [119]:
# | export


def replace_ipython_magics(code):
    # Replace Ipython magic and shell command symbol with comment
    code = code.replace("%", "#")
    code = re.sub(r"^!", "#", code)
    return re.sub(r"\n\W?!", "\n#", code)

In [120]:
throws = False
try:
    assert ast.parse(nb_cell_code)
except SyntaxError:
    throws = True
assert throws
assert type(ast.parse(replace_ipython_magics(nb_cell_code))) == ast.Module

In [121]:
# | export


def safe_div(numer, denom):
    return 0 if denom == 0 else numer / denom

In [122]:
assert safe_div(1, 1) == 1
assert safe_div(2, 1) == 2
assert safe_div(1, 2) == 0.5
assert safe_div(0, 1) == 0
assert safe_div(1, 0) == 0
assert safe_div(10, 1) == 10

In [123]:
# | export


def get_cell_code(nb):
    pnb = nbformat.from_dict(nb)
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in pnb.cells
            if c["cell_type"] == "code"
        ]
    )
    return nb_cell_code

# Potential Quality Indicators

## 1. Calls-per-Function

In [124]:
# | export


def calls_per_func(nb):
    nb_cell_code = get_cell_code(nb)
    func_defs = get_func_defs(nb_cell_code)
    func_calls = count_func_calls(nb_cell_code, func_defs)
    return func_calls

In [125]:
# | export


def mean_cpf(nb):
    return pd.Series(calls_per_func(nb)).mean()

In [126]:
# | export


def median_cpf(nb):
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", message="Mean of empty slice")
        return pd.Series(calls_per_func(nb)).median()

In [127]:
assert mean_cpf(nbdev_nb).round(2) == 2.23
assert median_cpf(nbdev_nb) == 1

In [128]:
assert mean_cpf(read_nb(nbdev_path)).round(2) == 2.23
assert mean_cpf(read_nb(nbdev_hq_path)).round(2) == 2.5
assert mean_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert mean_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.62
assert pd.isnull(mean_cpf(index))

In [129]:
assert median_cpf(read_nb(nbdev_path)) == 1.0
assert median_cpf(read_nb(nbdev_hq_path)).round(2) == 1.5
assert median_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert median_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.0
assert pd.isnull(median_cpf(index))

## 2. Asserts-to-Function Ratio

In [130]:
asserted_code = r"""
def something():
    pass; pass # in x 2
    
assert True

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
assert len(x) > 2
assert something() is None # something +1

def tr():
    return True
    
def get_seg(num):
    return 2
    
assert(tr)
assert(tr()) # tr +1
assert(tr() == 4) # tr +1
assert(4 ==tr()) # tr +1
assert 0 != 0
assert "' '".join(tr(1)) == "00" # tr +1
assert len(get_seg(50)) == 50 # get_seg +1
assert max([int(x) for x in get_seg(100)]) == 99 # get_seg +1
"""

In [131]:
import nbformat as nbf

In [132]:
asserted_nb = nbf.v4.new_notebook()
asserted_nb["cells"] = [nbf.v4.new_code_cell(asserted_code)]

In [133]:
# | export


def afr(nb):
    nb_cell_code = get_cell_code(nb)
    if nb_cell_code == "":  # no code cells - metric is not well defined
        return np.nan
    func_defs = get_func_defs(nb_cell_code)
    num_funcs = len(func_defs)

    assert_count = 0
    for stmt in ast.walk(ast.parse(nb_cell_code)):
        if isinstance(stmt, ast.Assert):
            assert_count += 1

    return safe_div(assert_count, num_funcs)

In [134]:
assert afr(nbdev_nb) > 1
assert afr(nbdev_hq_nb) > 1
assert afr(non_nbdev_nb) == 0
assert afr(non_nbdev_lq_nb) == 0
assert pd.isnull(afr(index))

## 3. In-line Asserts Per Function

In [135]:
# | export


def count_inline_asserts(code, func_defs):
    inline_func_asserts = Counter({k: 0 for k in func_defs})

    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Assert):
            for assert_st in ast.walk(stmt):
                if isinstance(assert_st, ast.Call):
                    func_name = (
                        assert_st.func.id
                        if "id" in assert_st.func.__dict__
                        else assert_st.func.attr
                    )
                    if func_name in inline_func_asserts:
                        inline_func_asserts[func_name] += 1
    return inline_func_asserts

In [136]:
# | export


def iaf(nb):
    nb_cell_code = get_cell_code(nb)
    if nb_cell_code == "":
        return np.nan
    func_defs = get_func_defs(nb_cell_code)
    return count_inline_asserts(nb_cell_code, func_defs)

In [137]:
func_defs = get_func_defs(asserted_code)
inline_asserts_expected = Counter(
    {"something": 1, "tr": 4, "get_seg": 2, "nb_to_sagemaker_pipeline": 0}
)
inline_asserts_actual = count_inline_asserts(asserted_code, func_defs)

In [138]:
assert inline_asserts_actual == inline_asserts_expected

In [139]:
assert 0.0 == pd.Series(iaf(nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(nbdev_hq_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_lq_nb)).median()
with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", message="Mean of empty slice")
    assert pd.isnull(pd.Series(iaf(index)).median())

In [140]:
iaf(non_nbdev_nb)

Counter({'scalar': 0, 'py_advanced': 0, 'pandas': 0})

In [141]:
iaf(non_nbdev_lq_nb)

Counter({'get_traffic_text': 0,
         'get_experiment_segment': 0,
         'evaluate': 0,
         'serve_num_topics': 0,
         'get_num_topics': 0,
         'get_topic_sizes': 0,
         'get_topics': 0,
         'plot_wordcloud': 0})

In [142]:
assert inline_asserts_expected == iaf(asserted_nb)

In [143]:
# | export


def mean_iaf(nb):
    return pd.Series(iaf(nb)).mean()

In [144]:
# | export


def median_iaf(nb):
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", message="Mean of empty slice")
        return pd.Series(iaf(nb)).median()

* What about calculating functions code coverage
* How does pytest-cov do it?

## 4. In-function Percentage

In [145]:
# | export


def calc_ifp(nb_cell_code):
    stmts_in_func = 0
    stmts_outside_func = 0
    for stmt in ast.walk(ast.parse(replace_ipython_magics(nb_cell_code))):
        if isinstance(stmt, ast.FunctionDef) and not stmt.name.startswith("_"):
            for body_item in stmt.body:
                stmts_in_func += 1
        elif isinstance(stmt, ast.Module):
            for body_item in stmt.body:
                if not isinstance(body_item, ast.FunctionDef):
                    stmts_outside_func += 1
    return (
        0
        if stmts_outside_func + stmts_in_func == 0
        else (stmts_in_func / (stmts_outside_func + stmts_in_func)) * 100
    )

In [146]:
assert (calc_ifp(nb_cell_code)) == (5 / (5 + 5)) * 100

In [147]:
# | export


def ifp(nb):
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in nb.cells
            if c["cell_type"] == "code"
        ]
    )
    if nb_cell_code == "":
        return np.nan
    return calc_ifp(nb_cell_code)

In [148]:
assert ifp(nbdev_nb) >= 0
assert ifp(nbdev_hq_nb) >= 0
assert ifp(non_nbdev_nb) >= 0
assert ifp(non_nbdev_lq_nb) >= 0
assert pd.isnull(ifp(index))

## 5. Markdown to Code Percent

In [149]:
# | export


def mcp(nb):
    md_cells = [c for c in nb.cells if c["cell_type"] == "markdown"]
    code_cells = [c for c in nb.cells if c["cell_type"] == "code"]
    num_code_cells = len(code_cells)
    if num_code_cells == 0:
        return np.nan
    num_md_cells = len(md_cells)
    return (
        100
        if num_code_cells == 0
        else (num_md_cells / (num_md_cells + num_code_cells)) * 100
    )

In [150]:
assert mcp(nbdev_nb) >= 0
assert mcp(nbdev_hq_nb) >= 0
assert mcp(non_nbdev_nb) >= 0
assert mcp(non_nbdev_lq_nb) >= 0
assert pd.isnull(mcp(index))

## 6. Total Code Length

In [151]:
# | export


def tcl(nb):
    return sum([len(c["source"]) for c in nb.cells if c["cell_type"] == "code"])

In [152]:
assert tcl(nbdev_nb) >= 50
assert tcl(nbdev_hq_nb) >= 50
assert tcl(non_nbdev_nb) >= 50
assert tcl(non_nbdev_lq_nb) >= 50

## 7. Sections to Code Len Ratio

In [153]:
# [coming soon] 

## Quality Indicators Map

> Add new quality indiactors here to be used. Signature contract is nb -> number. TODO: provide a proper typed signature, handle bools.

In [155]:
# | export

indicators = {
    "calls_per_func_mean": mean_cpf,
    "calls_per_func_median": median_cpf,
    "afr": afr,
    "iaf_mean": mean_iaf,
    "iaf_median": median_iaf,
    "in_func_perc": ifp,
    "markdown_code_percent": mcp,
    "total_code_len": tcl,
}


# Linting Functions

In [90]:
# rewrite
# create lint report from yaml file
# create warnings report from masks on lint report
# print markdown formatted report if issues - found

In [100]:
import yaml
lint_conf = yaml.safe_load(Path("scilint-default.yaml").read_text())

In [101]:
lint_conf

{'exclusions': None,
 'fail-over': 1,
 'csv-out-path': '/tmp/scilint.csv',
 'rounding-prec': 3,
 'print_syntax_errors': True,
 'warnings': {'lt': {'calls_per_func_median': 1,
   'calls_per_func_mean': 1,
   'in_func_perc': 20,
   'asserts_func_ratio': 1,
   'inline_asserts_per_func_median': 0,
   'inline_asserts_per_func_mean': 0.5,
   'markdown_code_percent': 5},
  'gt': {'total_code_len': 30000},
  'equals': {'syntax_errors': True}}}

## `lint_nb`

In [159]:
# | export


def lint_nb(
    nb_path: Path, lint_conf: Dict[str, Any], include_in_scoring: bool
):
    nb = read_nb(nb_path)

    has_syntax_error = False
    indic_vals = []
    try:
        for indic_name in lint_conf["indicators"]:
            indic_vals.append(round(indicators[indic_name](nb), lint_conf["precision"]))
    except SyntaxError as se:
        if print_syntax_errors:
            print(f"Syntax error in notebook: {nb_path} reason: ", se)
        has_syntax_error = True

    return tuple(indic_vals)

In [ ]:
# | export


# TODO generate and persist a new dataframe of warnings from this..


def format_quality_warning(metric, warning_data, warn_thresh, direction):
    for warning_row in warning_data.reset_index().itertuples():
        print(f'"{warning_row.index}" has: {metric} {direction} {warn_thresh}')

In [ ]:
# | export


def get_excluded_paths(paths: Iterable[Path], exclude_pattern: str):
    excl_paths = []
    for ex_pattern in exclude_pattern.split(","):
        ex_path = Path(ex_pattern)
        if ex_path.exists():
            excl_paths.extend([p for p in paths if ex_pattern in str(p)])
        elif not ex_path.exists():
            raise ValueError(f"Path component: {ex_path} does not exist")
        else:
            raise ValueError(
                f"Invalid exclusion pattern: {ex_path} pattern is comma separrated list of 'dir/' for directories and 'name.ipynb' for specific notebook"
            )
    return excl_paths

In [ ]:
paths = [Path(p) for p in nbglob(Path("."))]
assert sorted(
    [
        p.name
        for p in get_excluded_paths(paths, exclude_pattern="example_nbs/,index.ipynb")
    ]
) == sorted(
    [
        "non_nbdev_low_quality.ipynb",
        "nbdev_high_quality.ipynb",
        "non_nbdev.ipynb",
        "nbdev.ipynb",
        "index.ipynb",
        "syntax_error.ipynb",
    ]
)
assert sorted(
    [
        p.name
        for p in get_excluded_paths(
            paths, exclude_pattern="example_nbs/nbdev.ipynb,index.ipynb"
        )
    ]
) == sorted(["nbdev.ipynb", "index.ipynb"])

## `lint_nbs`

In [ ]:
# | export


def lint_nbs(
    lint_conf: Dict[str, Any]
):
    nb_paths = [Path(p) for p in nbglob(Path("."))]

    excluded_paths = None
    exclusions = lint_conf["exclusions"]
    if exclusions is not None:
        excluded_paths = get_excluded_paths(nb_paths, exclude_pattern=exclusions)

    results = []
    nb_names = []
    for nb_path in nb_paths:
        include_in_scoring = True
        if exclusions is not None:
            include_in_scoring = False if nb_path in excluded_paths else True

        nb_names.append(nb_path.stem)
        lint_result = lint_nb(nb_path, lint_conf, include_in_scoring)
        results.append(lint_result)

    lint_report = pd.DataFrame.from_records(
        data=results,
        index=nb_names,
        columns= indicators.keys() + ["include_in_scoring"],
    ).sort_values(["in_func_perc", "markdown_code_pct"], ascending=False)

    scoring_report = lint_report[lint_report.include_in_scoring].copy()
    num_warnings = calculate_warnings(
        scoring_report,
        lint_conf
    )

    lint_report.to_csv(csv_out_path)

    return lint_report, num_warnings

In [ ]:
# | export


def calculate_warnings(
    scoring_report: pd.DataFrame, lint_conf: Dict[str, Any]
):
    syntax_error_nbs = scoring_report[scoring_report.has_syntax_error].index.to_list()
    num_warnings = 0

    # TODO rewrite as masks
    for lt_metric_col, lt_metrics_threshold in zip(
        lt_metric_cols, lt_metrics_thresholds
    ):
        metrics_series = scoring_report[lt_metric_col]
        warning_data = metrics_series[metrics_series < lt_metrics_threshold]

        num_warnings += len(warning_data)
        format_quality_warning(
            lt_metric_col,
            warning_data,
            lt_metrics_threshold,
            direction="<",
        )
    for gt_metric_col, gt_metrics_threshold in zip(
        gt_metric_cols, gt_metrics_thresholds
    ):
        metrics_series = scoring_report[gt_metric_col]
        warning_data = metrics_series[metrics_series > gt_metrics_threshold]

        num_warnings += len(warning_data)
        format_quality_warning(
            gt_metric_col,
            warning_data,
            gt_metrics_threshold,
            direction=">",
        )

    num_warnings += len(syntax_error_nbs)
    # TODO separate calculation and display
    for syntax_error_nb in syntax_error_nbs:
        print(f'"{syntax_error_nb}" has syntax errors')
        
    return num_warnings

In [ ]:
lint_report, num_warnings = lint_nbs()
assert num_warnings == 6


*********************Begin Scilint Report*********************
"non_nbdev_low_quality" has: asserts_function_ratio < 1
"non_nbdev" has: asserts_function_ratio < 1
"non_nbdev_low_quality" has: iaf_mean < 0.5
"non_nbdev" has: iaf_mean < 0.5
"non_nbdev" has: markdown_code_pct < 5
"syntax_error" has syntax errors
*********************End Scilint Report***********************


In [ ]:
lint_report

,cpf_median,cpf_mean,in_function_pct,asserts_function_ratio,iaf_median,iaf_mean,markdown_code_pct,total_code_len,include_in_scoring,has_syntax_error
scilint,2.0,3.310,48.472,2.034,0.0,1.759,18.293,23662.0,True,False
nbdev_high_quality,1.5,2.500,44.118,1.667,0.0,1.000,30.769,4978.0,True,False
nbdev,1.0,2.231,50.725,1.308,0.0,0.846,30.769,4918.0,True,False
non_nbdev_low_quality,1.0,1.625,45.000,0.000,0.0,0.000,15.789,2955.0,True,False
non_nbdev,1.0,1.000,35.714,0.000,0.0,0.000,0.000,1233.0,True,False
index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,True,False
syntax_error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True


In [ ]:
_, num_warnings = lint_nbs(exclusions="""example_nbs/,index.ipynb""")
assert num_warnings == 0


*********************Begin Scilint Report*********************
No issues found
*********************End Scilint Report***********************


In [ ]:
_, num_warnings = lint_nbs(
    exclusions="""example_nbs/non_nbdev_low_quality.ipynb,index.ipynb,example_nbs/syntax_error.ipynb"""
)
assert num_warnings == 3


*********************Begin Scilint Report*********************
"non_nbdev" has: asserts_function_ratio < 1
"non_nbdev" has: iaf_mean < 0.5
"non_nbdev" has: markdown_code_pct < 5
*********************End Scilint Report***********************


In [ ]:
_, num_warnings = lint_nbs(exclusions="""example_nbs/non_nbdev_low_quality.ipynb""")
assert num_warnings == 4


*********************Begin Scilint Report*********************
"non_nbdev" has: asserts_function_ratio < 1
"non_nbdev" has: iaf_mean < 0.5
"non_nbdev" has: markdown_code_pct < 5
"syntax_error" has syntax errors
*********************End Scilint Report***********************


## `_lint`

In [ ]:
# | export

from typing import Dict, Any

def _lint(
    lint_config: Dict[Any, Any]
):
    lint_report, num_warnings = lint_nbs(lint_config)
    if fail_over == -1:
        print("Linting outcome ignored as fail_over set to -1")
    elif num_warnings > fail_over:
        print(
            f"Linting failed: total warnings ({num_warnings}) exceeded threshold ({fail_over})"
        )
        sys.exit(num_warnings)
    else:
        print("Linting succeeded")

## `_build`

In [ ]:
# | export


def _build(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: float = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
    print_syntax_errors: bool = False,
):
    print("Tidying notebooks..")
    tidy()
    if is_nbdev_project():
        nbdev_export.__wrapped__()
        print("Converted notebooks")
        print("Testing notebooks..")
        nbdev_test.__wrapped__()
    print("Running notebook linter..")
    _lint(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
        print_syntax_errors,
    )
    if is_nbdev_project():
        print("Cleaning notebooks..")
        nbdev_clean.__wrapped__()

# Console Scripts

## `scilint_tidy`

In [ ]:
# | export


@call_parse
def scilint_tidy():
    tidy()

## `scilint_lint`

In [ ]:
# | export


@call_parse
def scilint_lint(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: float = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
    print_syntax_errors: bool = False,
):
    _lint(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
    )

## `scilint_build`

In [ ]:
# | export


@call_parse
def scilint_build(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: float = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
):
    _build(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
    )

## `scilint_ci`

In [ ]:
# | export


@call_parse
def scilint_ci(
    cpf_med_warn_thresh: float = 1,
    cpf_mean_warn_thresh: float = 1,
    ifp_warn_thresh: float = 20,
    afr_warn_thresh: float = 1,
    iaf_med_warn_thresh: float = 0,
    iaf_mean_warn_thresh: float = 0.5,
    mcp_warn_thresh: float = 5,
    tcl_warn_thresh: int = 30000,
    rounding_precision: int = 3,
    csv_out_path: str = "/tmp/scilint.csv",
    exclusions: str = None,
    fail_over: int = 1,
):
    if not is_nbdev_project():
        print("scilint_ci feature is only available for nbdev projects")
        return

    _build(
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
        tcl_warn_thresh,
        rounding_precision,
        csv_out_path,
        exclusions,
        fail_over,
    )
    if not shutil.which("quarto"):
        print(
            "Quarto is not installed. A working quarto install is required for the CI build"
        )
        sys.exit(-1)
    nbdev_readme.__wrapped__()
    nbdev_docs.__wrapped__()